今までのlightBGMではスコアは頭打ち状態なので、他のアルゴリズムも試してみる。それらのアンサンブルでスコアの向上を狙ってみる。

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error

訓練データの読み込み

In [21]:
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange_scores = pd.read_csv('room_arrange_scores.csv')
contract_span = pd.read_csv('contract_span.csv')
floor_scores = pd.read_csv('floor_scores.csv')
Floor_scores = pd.read_csv('capital_floor_scores.csv')
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')

rent = pd.read_csv('rent.csv')

テストデータ

In [22]:
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange_scores = pd.read_csv('test_room_arrange_scores.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')

In [47]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'rf',#ランダムフォレストを使う
    'metrics':'rmse',
    'bagging_freq': 10,
    'bagging_fraction': 0.5,
    'reg_lambda': 3.681194978110037e-06,
    'max_bin': 522,
    'num_leaves': 124
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001577 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2444
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35472.1	valid_1's rmse: 41641.7
[20]	training's rmse: 32771.7	valid_1's rmse: 39412.2
[30]	training's rmse: 31925.3	valid_1's rmse: 38847
[40]	training's rmse: 31901.6	valid_1's rmse: 39228.9
Early stopping, best iteration is:
[30]	training's rmse: 31925.3	valid_1's rmse: 38847


あまり芳しくない値なので、パラメータチューニングを行ってみる。

In [61]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

def objective(trial):
    category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    params = {
        'objective':'regression',
        'boosting_type':'rf',
        'metrics':'rmse',
        'learning_rate':0.05,
        'bagging_freq':1,
        'bagging_fraction': trial.suggest_float('reg_lambda', 0.21, 0.99, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
        'max_bin': trial.suggest_int('max_bin', 255, 600),
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        #verbose_eval=10, 
                        num_boost_round=3000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

    loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
    return loss

In [ ]:
optuna.logging.disable_default_handler()
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=200)
study.best_params

このチューニングされたパラメータを使う。

In [57]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'rf',
    'metrics':'rmse',
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'reg_lambda': 0.8541733691883097,
    'max_bin': 510,
    'num_leaves': 119
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001696 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2420
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 29013.2	valid_1's rmse: 36036.6
Early stopping, best iteration is:
[8]	training's rmse: 29070.4	valid_1's rmse: 35940.1


/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

ランダムフォレストではうまくいかなそう。他のアルゴリズムを使う。

dartを使う。

In [63]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'dart',
    'metrics':'rmse',
    'lambda_l1':0.000001
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001002 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1877
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 40839.2	valid_1's rmse: 45284.8
[20]	training's rmse: 26245.4	valid_1's rmse: 33594.9
[30]	training's rmse: 20647.8	valid_1's rmse: 29474.7
[40]	training's rmse: 19195.2	valid_1's rmse: 28396.9
[50]	training's rmse: 23317.3	valid_1's rmse: 31380.5
[60]	training's rmse: 27004.9	valid_1's rmse: 34222.3
[70]	training's rmse: 25752.1	valid_1's rmse: 33259.3
[80]	training's rmse: 21192.3	valid_1's rmse: 29947.9
[90]	training's rmse: 21533.4	valid_1's rmse: 30263.4
[100]	training's rmse: 22276.4	valid_1's rmse: 30863.2
[110]	training's rmse: 28096	valid_1's rmse: 

dartはよさそう。チューニングしてみる。

In [65]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

def objective(trial):
    category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    params = {
        'objective':'regression',
        'boosting_type':'dart',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
        'max_bin': trial.suggest_int('max_bin', 255, 600),
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=3000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

    loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
    return loss

以下のチューニングは80分以上かかるので注意

In [66]:
optuna.logging.disable_default_handler()
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=50)
study.best_params

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003517 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2404
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 52663.9	valid_1's rmse: 55365.7
[20]	training's rmse: 39657.6	valid_1's rmse: 44316.7
[30]	training's rmse: 31838.6	valid_1's rmse: 38205.8
[40]	training's rmse: 28519.3	valid_1's rmse: 35667.6
[50]	training's rmse: 31848.8	valid_1's rmse: 38332.3
[60]	training's rmse: 35992.3	valid_1's rmse: 41896.1
[70]	training's rmse: 37909.6	valid_1's rmse: 43613
[80]	training's rmse: 32864.7	valid_1's rmse: 39352.6
[90]	training's rmse: 32710.1	valid_1's rmse: 39197.6
[100]	training's rmse: 34820.4	valid_1's rmse: 41039.9
[110]	training's rmse: 41580.9	valid_1's rmse: 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000604 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2174
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 52622.8	valid_1's rmse: 55414.6
[20]	training's rmse: 39627	valid_1's rmse: 44482.8
[30]	training's rmse: 31876.7	valid_1's rmse: 38415.4
[40]	training's rmse: 28477.7	valid_1's rmse: 35814.1
[50]	training's rmse: 31836.2	valid_1's rmse: 38484.5
[60]	training's rmse: 35907.7	valid_1's rmse: 41994.5
[70]	training's rmse: 37924.2	valid_1's rmse: 43747.6
[80]	training's rmse: 32918.6	valid_1's rmse: 39553.6
[90]	training's rmse: 32791.6	valid_1's rmse: 39384.1
[100]	training's rmse: 34829.3	valid_1's rmse: 41121.4
[110]	training's rmse: 41619.8	valid_1's rmse: 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001272 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2396
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 53407.9	valid_1's rmse: 56006.9
[20]	training's rmse: 40193	valid_1's rmse: 44980.1
[30]	training's rmse: 32795.4	valid_1's rmse: 39012.2
[40]	training's rmse: 29718.6	valid_1's rmse: 36545.7
[50]	training's rmse: 32784.6	valid_1's rmse: 39011.2
[60]	training's rmse: 36744.3	valid_1's rmse: 42404.7
[70]	training's rmse: 38578.9	valid_1's rmse: 43975.3
[80]	training's rmse: 33675.6	valid_1's rmse: 39784.5
[90]	training's rmse: 33540.3	valid_1's rmse: 39617.1
[100]	training's rmse: 35399.3	valid_1's rmse: 41222.2
[110]	training's rmse: 42097.2	valid_1's rmse: 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001180 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2404
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 52459.3	valid_1's rmse: 55169.2
[20]	training's rmse: 39435.5	valid_1's rmse: 44195.9
[30]	training's rmse: 31465	valid_1's rmse: 37920.1
[40]	training's rmse: 27996.2	valid_1's rmse: 35280.5
[50]	training's rmse: 31403.9	valid_1's rmse: 37966.2
[60]	training's rmse: 35716.8	valid_1's rmse: 41616.3
[70]	training's rmse: 37594	valid_1's rmse: 43265
[80]	training's rmse: 32691.9	valid_1's rmse: 39217.8
[90]	training's rmse: 32625.7	valid_1's rmse: 39077.1
[100]	training's rmse: 34652.6	valid_1's rmse: 40780
[110]	training's rmse: 41409.4	valid_1's rmse: 46528.

{'reg_lambda': 2.1600820741402023e-05, 'max_bin': 556, 'num_leaves': 126}

チューニングの結果を実際に使う。

In [82]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'dart',
    'metrics':'rmse',
    'reg_lambda': 2.1600820741402023e-05,
    'max_bin': 556,
    'num_leaves': 126
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=1000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001447 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2512
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Start training from score 118651.337373
[10]	training's rmse: 38943.1	valid_1's rmse: 44156.4
[20]	training's rmse: 23716.7	valid_1's rmse: 32574.6
[30]	training's rmse: 17910.6	valid_1's rmse: 28302.2
[40]	training's rmse: 16671	valid_1's rmse: 27385.8
[50]	training's rmse: 21191	valid_1's rmse: 30425
[60]	training's rmse: 25068.6	valid_1's rmse: 33244.9
[70]	training's rmse: 23943.7	valid_1's rmse: 32352.9
[80]	training's rmse: 18952.1	valid_1's rmse: 28768.6
[90]	training's rmse: 19363.8	valid_1's rmse: 29087.8
[100]	training's rmse: 20278.5	valid_1's rmse: 29698.8
[110]	training's rmse: 26486.3	valid_1's rmse: 3429

予測結果をファイル出力

In [83]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('DART_result.csv',index=False)

次に、gossを使ってみる。

In [73]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'goss',
    'metrics':'rmse',
    'reg_lambda': 2.1600820741402023e-05,
    'max_bin': 556,
    'num_leaves': 126
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=1000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001224 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2512
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35304	valid_1's rmse: 41369.9
[20]	training's rmse: 22977.2	valid_1's rmse: 32170.1
[30]	training's rmse: 18041.5	valid_1's rmse: 28717
[40]	training's rmse: 15809	valid_1's rmse: 27219.7
[50]	training's rmse: 14337	valid_1's rmse: 26251.7
[60]	training's rmse: 13343.7	valid_1's rmse: 25612.1
[70]	training's rmse: 12530.2	valid_1's rmse: 25222
[80]	training's rmse: 11831	valid_1's rmse: 24868.1
[90]	training's rmse: 11255.6	valid_1's rmse: 24616.2
[100]	training's rmse:

これも悪くはなさそう。

In [74]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

def objective(trial):
    category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    params = {
        'objective':'regression',
        'boosting_type':'goss',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
        'max_bin': trial.suggest_int('max_bin', 255, 600),
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=3000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

    loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
    return loss

In [75]:
optuna.logging.disable_default_handler()
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=50)
study.best_params

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001350 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2404
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 50091.9	valid_1's rmse: 53168.8
[20]	training's rmse: 36314.4	valid_1's rmse: 41684.4
[30]	training's rmse: 28247.6	valid_1's rmse: 35412.2
[40]	training's rmse: 23385.5	valid_1's rmse: 31856.5
[50]	training's rmse: 20301.6	valid_1's rmse: 29676.4
[60]	training's rmse: 18424.6	valid_1's rmse: 28342.4
[70]	training's rmse: 17099.7	valid_1's rmse: 27389.4
[80]	training's rmse: 16223.9	valid_1's rmse: 26804.6
[90]	training's rmse: 15490.4	valid_1's rmse: 26348.9
[100]	trai

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000577 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2174
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 50032.4	valid_1's rmse: 53157.5
[20]	training's rmse: 36299.3	valid_1's rmse: 41856.8
[30]	training's rmse: 28423.8	valid_1's rmse: 35764.1
[40]	training's rmse: 23285	valid_1's rmse: 32057.5
[50]	training's rmse: 20196.5	valid_1's rmse: 29796.2
[60]	training's rmse: 18339.4	valid_1's rmse: 28454.1
[70]	training's rmse: 17097.4	valid_1's rmse: 27506.3
[80]	training's rmse: 16166.1	valid_1's rmse: 26842.1
[90]	training's rmse: 15460.4	valid_1's rmse: 26408.2
[100]	traini

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000768 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2396
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 50893.9	valid_1's rmse: 53871
[20]	training's rmse: 36873.4	valid_1's rmse: 42244.1
[30]	training's rmse: 29458.6	valid_1's rmse: 36180.4
[40]	training's rmse: 24499.7	valid_1's rmse: 32340.5
[50]	training's rmse: 21439.7	valid_1's rmse: 29969.5
[60]	training's rmse: 19474.1	valid_1's rmse: 28487.5
[70]	training's rmse: 18282.3	valid_1's rmse: 27523.7
[80]	training's rmse: 17416.2	valid_1's rmse: 26931.5
[90]	training's rmse: 16827.4	valid_1's rmse: 26461
[100]	training

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001157 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2404
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 49818.1	valid_1's rmse: 52919.1
[20]	training's rmse: 35844.3	valid_1's rmse: 41329.7
[30]	training's rmse: 28075	valid_1's rmse: 35260.2
[40]	training's rmse: 23192.6	valid_1's rmse: 31652.1
[50]	training's rmse: 20006.6	valid_1's rmse: 29414.1
[60]	training's rmse: 18114.7	valid_1's rmse: 28059.4
[70]	training's rmse: 16825.3	valid_1's rmse: 27146.2
[80]	training's rmse: 15882.4	valid_1's rmse: 26505.2
[90]	training's rmse: 15115	valid_1's rmse: 26015.7
[100]	training

{'reg_lambda': 1.2667345328336822e-06, 'max_bin': 405, 'num_leaves': 118}

さっそくチューニングをしてみる。

In [80]:
X_train = pd.concat([house_age, area_size, contract_span, floor_scores, Floor_scores, stations, minits, addresses, room_arrange], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age, test_area_size, test_contract_span, test_floor_scores, test_Floor_scores, test_stations, test_minits, test_addresses, test_room_arrange], axis=1)

category_lists = ['最寄り駅', '所在地', 'L', 'D', 'K', 'S']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'boosting_type':'goss',
    'metrics':'rmse',
    'reg_lambda': 1.2667345328336822e-06,
    'max_bin': 405,
    'num_leaves': 118
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=1000, 
                    early_stopping_rounds=10,
                    categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument i

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001485 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2186
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 13
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 35423.6	valid_1's rmse: 41182.2
[20]	training's rmse: 23148.4	valid_1's rmse: 32080.4
[30]	training's rmse: 18293.5	valid_1's rmse: 28600.7
[40]	training's rmse: 15959.5	valid_1's rmse: 27021
[50]	training's rmse: 14662.6	valid_1's rmse: 26344.8
[60]	training's rmse: 13679.4	valid_1's rmse: 25769.8
[70]	training's rmse: 12864.6	valid_1's rmse: 25331.6
[80]	training's rmse: 12214	valid_1's rmse: 25019
[90]	training's rmse: 11655.3	valid_1's rmse: 24792
[100]	training's r

In [81]:
df = pd.read_csv('test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('GOSS_result.csv',index=False)